In [17]:
import serial
import time

# Set up the serial connection
ser = serial.Serial(port='COM8', baudrate=9600, parity=serial.PARITY_NONE,
                    stopbits=serial.STOPBITS_ONE, bytesize=serial.EIGHTBITS)

try:
    while True:
        if ser.readable():
            res = ser.readline()
            # Decode and print the line received from Arduino
            print(res.decode().strip())
        time.sleep(0.1)  # Sleep to give some buffer time between reads
except KeyboardInterrupt:
    print("Stopped by user")
    ser.close()  # Close the serial connection when done

OV767X Camera Capture

Camera settings:
width = 176
height = 144
bits per pixel = 16

Send the 'c' character to read a frame ...

Reading frame

... frame

... frame

... frame

... frame

... frame

... frame

... frame

... frame

Stopped by user


In [11]:
buffer=""
while True:
   oneByte = f.read(1)
   if oneByte == b"\r":    #byte단위로 read, 구분자 '\r'
       break
   else:
       buffer += oneByte.decode()

print (buffer.strip())


NameError: name 'f' is not defined

In [3]:
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from PIL import Image, ImageDraw
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

# you can specify the revision tag if you don't want the timm dependency
processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", revision="no_timm")

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)

# convert outputs (bounding boxes and class logits) to COCO API
# let's only keep detections with score > 0.9
target_sizes = torch.tensor([image.size[::-1]])
results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
            f"Detected {model.config.id2label[label.item()]} with confidence "
            f"{round(score.item(), 3)} at location {box}"
    )


Detected remote with confidence 0.998 at location [40.16, 70.81, 175.55, 117.98]
Detected remote with confidence 0.996 at location [333.24, 72.55, 368.33, 187.66]
Detected couch with confidence 0.995 at location [-0.02, 1.15, 639.73, 473.76]
Detected cat with confidence 0.999 at location [13.24, 52.05, 314.02, 470.93]
Detected cat with confidence 0.999 at location [345.4, 23.85, 640.37, 368.72]


In [4]:
# After you have your results from the model:
draw = ImageDraw.Draw(image)
for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = box.tolist()
    draw.rectangle([(box[0], box[1]), (box[2], box[3])], outline="red", width=3)
    draw.text((box[0], box[1]), model.config.id2label[label.item()], fill="red")

image.show()

In [7]:
!pip install pyinstaller


   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
    --------------------------------------- 0.0/1.3 MB 1.3 MB/s eta 0:00:01
   ---- ----------------------------------- 0.1/1.3 MB 2.1 MB/s eta 0:00:01
   ---------- ----------------------------- 0.3/1.3 MB 3.5 MB/s eta 0:00:01
   ---------------------- ----------------- 0.7/1.3 MB 5.1 MB/s eta 0:00:01
   ---------------------------------------  1.3/1.3 MB 7.5 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 7.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/71.8 kB ? eta -:--:--
   ---------------------------------------- 71.8/71.8 kB 3.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/329.0 kB ? eta -:--:--
   ---------------------------------------- 329.0/329.0 kB ? eta 0:00:00


In [9]:
!pyinstaller --onefile your_script.py

385 INFO: PyInstaller: 6.5.0, contrib hooks: 2024.3
385 INFO: Python: 3.9.19 (conda)
391 INFO: Platform: Windows-10-10.0.22631-SP0
Script file 'your_script.py' does not exist.
